In [1]:
import zarr
import xarray as xr
import time
import dask



In [2]:
ask_workers=10
memory='5GB'
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import dask.dataframe as dd

cluster = PBSCluster(cores=1, memory=memory, project='PerfTestPangeo', walltime='04:00:00')
cluster.scale(ask_workers)

c = Client(cluster)

c

/home/ad/alberta/.conda/envs/perf-pangeo/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42102 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://10.120.43.29:46632 Dashboard: http://10.120.43.29:42102/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
from dask.utils import ensure_dict, format_bytes
    
wk = c.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


Workers= 10, Cores=10, Memory=50.00 GB


In [4]:
%time ds=xr.open_zarr('/work/ALT/odatis/eNATL60/zarr/eNATL60-BLBT02-SSH-1h')

CPU times: user 93.9 ms, sys: 18.7 ms, total: 113 ms
Wall time: 150 ms


In [5]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 218 ms, sys: 8.17 ms, total: 226 ms
Wall time: 257 ms


In [ ]:
%time mean.load()

In [7]:
c.close()
cluster.close()